In [1]:
import torch, os
torch.manual_seed(0) 
import warnings;warnings.filterwarnings("ignore")
from HINT.dataloader import csv_three_feature_2_dataloader, generate_admet_dataloader_lst
from HINT.molecule_encode import MPNN, ADMET
from HINT.icdcode_encode import GRAM, build_icdcode2ancestor_dict
from HINT.protocol_encode import Protocol_Embedding
from HINT.model import HINTModel 
device = torch.device("cpu")  ## cuda:0
if not os.path.exists("figure"):
    os.makedirs("figure")
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
### data-provider datasets
base_name = 'phase_I' ### 'toy', 'phase_I', 'phase_II', 'phase_III', 'indication'
datafolder = "data" ### '/data/clinical-trial-outcome-prediction/data/'
train_file = os.path.join(datafolder, base_name + '_train.csv')
valid_file = os.path.join(datafolder, base_name + '_valid.csv')
test_file = os.path.join(datafolder, base_name + '_test.csv')

train_loader = csv_three_feature_2_dataloader(train_file, shuffle=True, batch_size=32) 
valid_loader = csv_three_feature_2_dataloader(valid_file, shuffle=False, batch_size=32) 
test_loader = csv_three_feature_2_dataloader(test_file, shuffle=False, batch_size=32) 

In [3]:
# ### Leo's datasets
# base_name = 'phase_II_cancer' ### 'cancer', 'digest', 'infection', 'nervous', 'respiratory'
# datafolder = "/data/clinical-trial-outcome-prediction/data/"
# train_file = os.path.join(datafolder, base_name + '_train.csv')
# valid_file = os.path.join(datafolder, base_name + '_valid.csv')
# test_file = os.path.join(datafolder, base_name + '_test.csv')

# train_loader = csv_three_feature_2_dataloader(train_file, shuffle=True, batch_size=32) 
# valid_loader = csv_three_feature_2_dataloader(valid_file, shuffle=False, batch_size=32) 
# test_loader = csv_three_feature_2_dataloader(test_file, shuffle=False, batch_size=32) 

In [3]:
mpnn_model = MPNN(mpnn_hidden_size = 50, mpnn_depth=3, device = device)
icdcode2ancestor_dict = build_icdcode2ancestor_dict()
gram_model = GRAM(embedding_dim = 50, icdcode2ancestor = icdcode2ancestor_dict, device = device)
protocol_model = Protocol_Embedding(output_dim = 50, highway_num=3, device = device)

molecule_encoder = mpnn_model
disease_encoder = gram_model
protocol_encoder = protocol_model

In [4]:
def get_embed(dataloader):
    nctid_lst, label_lst, smiles_lst2, icdcode_lst3, criteria_lst = [], [], [], [], []
    for nctid, label, smiles, icdcode, criteria in dataloader:
        nctid_lst.extend(nctid)
        label_lst.extend([i.item() for i in label])
        smiles_lst2.extend(smiles)
        icdcode_lst3.extend(icdcode)
        criteria_lst.extend(criteria)
        
    molecule_embed = molecule_encoder.forward_smiles_lst_lst(smiles_lst2)
    icd_embed = disease_encoder.forward_code_lst3(icdcode_lst3)
    protocol_embed = protocol_encoder.forward(criteria_lst)
    print(molecule_embed.shape, icd_embed.shape, protocol_embed.shape)
    return molecule_embed, icd_embed, protocol_embed

In [5]:
def preprocess(file, loader):
    df = pd.read_csv(file)
    df.drop(['phase', 'why_stop'], axis=1, inplace=True)
    df.drop(['icdcodes', 'smiless', 'criteria'], axis=1, inplace=True)
    df.drop(['diseases', 'drugs'], axis=1, inplace=True) ## FE later
    
    molecule_embed, icd_embed, protocol_embed = get_embed(loader)

    molecule_df = pd.DataFrame(molecule_embed.detach().numpy(), columns=[f'molecule_feature_{i}' for i in range(len(molecule_embed[0]))])
    icd_df = pd.DataFrame(icd_embed.detach().numpy(), columns=[f'icd_feature_{i}' for i in range(len(icd_embed[0]))])
    protocol_df = pd.DataFrame(protocol_embed.detach().numpy(), columns=[f'protocol_feature_{i}' for i in range(len(protocol_embed[0]))])

    df = pd.concat([df, molecule_df, icd_df, protocol_df], axis=1)
    return df

In [6]:
train_df = preprocess(train_file, train_loader)
valid_df = preprocess(valid_file, valid_loader)
test_df = preprocess(test_file, test_loader)
print(train_df.shape, valid_df.shape, test_df.shape)

torch.Size([1044, 50]) torch.Size([1044, 50]) torch.Size([1044, 50])
torch.Size([117, 50]) torch.Size([117, 50]) torch.Size([117, 50])
torch.Size([627, 50]) torch.Size([627, 50]) torch.Size([627, 50])
(1044, 153) (117, 153) (627, 153)


In [7]:
X_train, y_train = train_df.drop(['nctid','label'], axis=1), train_df['label']  
X_valid, y_valid = valid_df.drop(['nctid','label'], axis=1), valid_df['label']  
X_test, y_test = test_df.drop(['nctid','label'], axis=1), test_df['label']  
print(X_train.shape, X_valid.shape, X_test.shape)

(1044, 151) (117, 151) (627, 151)


In [10]:
X_test.isnull().sum().sort_values(ascending=False)  # no missing data

status                 0
icd_feature_44         0
icd_feature_46         0
icd_feature_47         0
icd_feature_48         0
                      ..
icd_feature_0          0
icd_feature_1          0
icd_feature_2          0
icd_feature_3          0
protocol_feature_49    0
Length: 151, dtype: int64

In [33]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_recall_curve, precision_score, recall_score, auc

start_time = time.time()
# model = CatBoostClassifier(iterations=100, random_state=42)
model = CatBoostClassifier(random_state=42)
hyperparameters = {
    'depth': [3, 4, 5, 6, 7],
    'l2_leaf_reg': [1, 3, 5],
    'iterations': [100, 200]
}
grid_search = GridSearchCV(model, hyperparameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train, cat_features=[0], verbose=0)
# model.fit(X_train, y_train, cat_features=[0])
print("time is ", time.time()-start_time)

time is  264.83298087120056


In [34]:
best_model = grid_search.best_estimator_
best_params = best_model.get_params()
print(best_params)

{'iterations': 200, 'depth': 5, 'l2_leaf_reg': 1, 'random_state': 42}


In [35]:
from sklearn.metrics import accuracy_score
model = grid_search.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_valid = model.predict(X_valid)
y_pred_test = model.predict(X_test)
print(accuracy_score(y_pred_train, y_train))
print(accuracy_score(y_pred_valid, y_valid))
print(accuracy_score(y_pred_test, y_test))

0.7828054298642534
0.7703488372093024
0.8385689354275742


In [13]:
def print_metrics(y_true, y_pred, label):
    print(f"{label} ROC AUC: {round(roc_auc_score(y_true, y_pred),3)}")
    print(f"{label} F1: {round(f1_score(y_true, y_pred),3)}")
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    print(f"{label} PR-AUC: {round(auc(recall, precision),3)}")
    print(f"{label} Precision: {round(precision_score(y_true, y_pred),3)}")
    print(f"{label} Recall: {round(recall_score(y_true, y_pred),3)}")
    print(f"{label} Accuracy: {round(accuracy_score(y_true, y_pred),3)}")
    print(f"{label} Predict 1 ratio: {round(sum(y_pred) / len(y_pred),3)}")
    print(f"{label} Label 1 ratio: {round(sum(y_true) / len(y_true),3)}")

# print_metrics(y_train, y_pred_train, 'Train')
# print_metrics(y_valid, y_pred_valid, 'Valid')
print_metrics(y_test, y_pred_test, 'Test')

Test ROC AUC: 0.832
Test F1: 0.854
Test PR-AUC: 0.89
Test Precision: 0.839
Test Recall: 0.87
Test Accuracy: 0.836
Test Predict 1 ratio: 0.574
Test Label 1 ratio: 0.553


In [52]:
### LightGBM

In [8]:
X_train['status'] = X_train['status'].astype('category')
X_valid['status'] = X_valid['status'].astype('category')
X_test['status'] = X_test['status'].astype('category')

In [16]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_recall_curve, precision_score, recall_score, auc

start_time = time.time()
model = lgb.LGBMClassifier(n_estimators=100, random_state=42, verbose_eval=-1)
# model = lgb.LGBMClassifier(random_state=42)
# hyperparameters = {
#     'learning_rate': [0.05, 0.1, 0.15],
#     'max_depth': [3, 4, 5, 6, 7],
#     'num_leaves': [31, 63],
#     'n_estimators': [100, 200]
# }
# grid_search = GridSearchCV(model, hyperparameters, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)
model.fit(X_train, y_train)
print("time is ", time.time()-start_time)

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Number of positive: 592, number of negative: 452
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000856 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23418
[LightGBM] [Info] Number of data points in the train set: 1044, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.567050 -> initscore=0.269824
[LightGBM] [Info] Start training from score 0.269824
time is  0.3997180461883545


In [10]:
best_model = grid_search.best_estimator_
best_params = best_model.get_params()
print(best_params)

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': 3, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


In [17]:
# model = grid_search.best_estimator_
y_pred_train = model.predict(X_train)
y_pred_valid = model.predict(X_valid)
y_pred_test = model.predict(X_test)
print(accuracy_score(y_pred_train, y_train))
print(accuracy_score(y_pred_valid, y_valid))
print(accuracy_score(y_pred_test, y_test))

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
0.9894636015325671
0.7863247863247863
0.8437001594896332


In [18]:
# print_metrics(y_train, y_pred_train, 'Train')
# print_metrics(y_valid, y_pred_valid, 'Valid')
print_metrics(y_test, y_pred_test, 'Test')

Test ROC AUC: 0.839
Test F1: 0.862
Test PR-AUC: 0.895
Test Precision: 0.843
Test Recall: 0.882
Test Accuracy: 0.844
Test Predict 1 ratio: 0.579
Test Label 1 ratio: 0.553
